Połączenie z dyskiem 

In [ ]:
import os
import shutil
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pathlib
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
import datetime
import json
from contextlib import redirect_stdout

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Przygotowywanie danych

In [ ]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

dirName='/content/drive/MyDrive/UM_baza_danych/50gr'
listOfFiles = getListOfFiles(dirName)
counter = 0
width_resized = 128
height_resized = 128
dim = (width_resized, height_resized)

for file in listOfFiles:
  img = cv2.imread(file)
  counter += 1
  height, width, channels = img.shape
  img = img[int(height/3.5):int(height-height/3.5), int(width/3.5):int(width-width/3.5)]
  resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
  cv2.imwrite('/content/drive/MyDrive/UM_baza_danych/resized/50gr/' + os.path.basename(file), resized)
  print(counter)

Pobierz set zdjęć

In [ ]:
! cp -r /content/drive/MyDrive/UM_baza_danych/resized /home

Pobierz set zdjęć testowych

In [ ]:
! cp -r /content/drive/MyDrive/UM_baza_danych/RandomCoins /home

Obróbka zdjęć


In [ ]:
batch_size = 32
img_height = 128
img_width = 128
epochs = 30
seed = 123

def train_data():
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=seed,
        shuffle=False,
        image_size=(img_height, img_width),
        batch_size=batch_size)
    return train_ds


def val_data():
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=seed,
        shuffle=False,
        image_size=(img_height, img_width),
        batch_size=batch_size)
    return val_ds

def plot_val_train(epochs, history):

    if (type(history) is dict):

      acc = history['accuracy']
      val_acc = history['val_accuracy']

      loss = history['loss']
      val_loss = history['val_loss']
   
    else:

      acc = history.history['accuracy']
      val_acc = history.history['val_accuracy']

      loss = history.history['loss']
      val_loss = history.history['val_loss']
    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.axhline(y = 0.994, color = 'r', linestyle = '-', label = "99,4%")
    plt.legend(loc='right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

def save_val_train(epochs, history, model, date, number_of_filters):

    architecture = ''
    for layer in model.layers:
      architecture += layer.name + "__"

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.savefig('/content/drive/MyDrive/UM_baza_danych/{}/'.format(date) + str(architecture) + str(number_of_filters) + '.png')

def plot_starting_image():
    plt.figure(figsize=(10, 10))
    for images, labels in train_ds.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(class_names[labels[i]])
            plt.axis("off")

images_classes_path = '/home/resized'

data_dir = pathlib.Path(images_classes_path)
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

NAME = "COINS-CNN"
train_ds = train_data()
val_ds = val_data()
class_names = train_ds.class_names
val_batches = tf.data.experimental.cardinality(val_ds)
test_dataset = val_ds.take(val_batches // 5)
validation_dataset = val_ds.skip(val_batches // 5)



Trenowanie modelu 

In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

num_classes = 4
epochs = 30
number_of_filters = 16
model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1. / 255),
    layers.Conv2D(number_of_filters, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(number_of_filters*2, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(number_of_filters*4, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(number_of_filters*4, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

tensorboard = TensorBoard(log_dir="/home/logs/{}".format(NAME))

with tf.device('/device:GPU:0'):
  opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(
      optimizer=opt,
      loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])

  history = model.fit(
      train_ds,
      validation_data=validation_dataset,
      epochs=epochs
  )

t = datetime.datetime.now()
export_path = "/content/drive/MyDrive/UM_baza_danych/{}".format(t)
history_dict = history.history

model.save(export_path + '/weights')
json.dump(history_dict, open(export_path + '/history', 'w'))
save_val_train(epochs, history, model, t, number_of_filters)

model.summary()
with open(export_path + '/model.txt', 'w') as f:
  with redirect_stdout(f):
    model.summary()



Załaduj model

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/UM_baza_danych/2021-05-13 18:26:23.174296_4conv_128/weights")
history = json.load(open("/content/drive/MyDrive/UM_baza_danych/2021-05-13 18:26:23.174296_4conv_128/history", 'r'))

In [ ]:
model.summary()

Pokaż przykładowe obrazy przed treningiem

In [ ]:
plot_starting_image()

Pokaż wykresy rezultatów

In [ ]:
plot_val_train(epochs,history)

Sprawdź model i źle zaklasyfikowane zdjęcia

In [ ]:
loss, accuracy = model.evaluate(validation_dataset)
print('Test accuracy :', accuracy)

a = 0
img_count = 0
errors = 0
max_score_correct = 0
min_score_correct = 1
max_score_false = 0
min_score_false = 1
plt.figure(figsize=(40, 40))

for i in range(tf.data.experimental.cardinality(validation_dataset)):
  image_batch, label_batch = validation_dataset.as_numpy_iterator().next()
  predictions = model.predict(image_batch)
  for j in range(batch_size):
    img_count += 1
    score = tf.nn.softmax(predictions[j])

    if(label_batch[j] == np.argmax(score)):
      if (np.max(score) > max_score_correct):
        max_score_correct = np.max(score)
      if (np.max(score) < min_score_correct):
        min_score_correct = np.max(score)

    if(label_batch[j] != np.argmax(score)):
      errors += 1
      if (np.max(score) > max_score_false):
        max_score_false = np.max(score)
      if (np.max(score) < min_score_false):
        min_score_false = np.max(score)
      

      ax = plt.subplot(10, 10, a + 1)
      plt.imshow(image_batch[j].astype("uint8"))
      plt.title(class_names[np.argmax(score)])
      a += 1

print('Test accuracy : {}%'.format(errors/img_count*100))
print("Maksymalna pewność modelu dla poprawnej klasyfikacji: ", max_score_correct)
print("Minimalna pewność modelu dla poprawnej klasyfikacji: ", min_score_correct)
print("Maksymalna pewność modelu dla błędnej klasyfikacji: ", max_score_false)
print("Minimalna pewność modelu dla błędnej klasyfikacji: ", min_score_false)
print(img_count)
print(errors)


Testowanie wytrenowanego modelu na losowych zdjęciach spoza bazy

In [ ]:
plt.figure(figsize=(16, 16))
j = 0;
for i in os.listdir('/home/RandomCoins'):
    ax = plt.subplot(4, 4, j + 1)
    img = tf.keras.preprocessing.image.load_img('/home/RandomCoins/'+i, target_size=(128, 128))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    img = tf.keras.preprocessing.image.load_img('/home/RandomCoins/'+i, target_size=(512, 512))
    plt.imshow(img)
    plt.title(class_names[np.argmax(score)])
    j+=1


In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import preprocessing
from tensorflow.keras import backend as K
from tensorflow.keras import models
import cv2
from google.colab.patches import cv2_imshow # cv2.imshow does not work on Google Colab notebooks, which is why we are using cv2_imshow instead
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def do_heat_map(img_user, flag):

  if (flag == "src"):
    image_src = img_user
    # Load pre-trained Keras model and the image to classify
    image = tf.keras.preprocessing.image.load_img(image_src, target_size=(128, 128))
  else:
    image = img_user
  img_tensor = preprocessing.image.img_to_array(image)
  img_tensor = np.expand_dims(img_tensor, axis=0)
  img_tensor = preprocess_input(img_tensor)

  conv_layer = model.get_layer("conv2d_7")
  heatmap_model = models.Model([model.inputs], [conv_layer.output, model.output])

  # Get gradient of the winner class w.r.t. the output of the (last) conv. layer
  with tf.GradientTape() as gtape:
      conv_output, predictions = heatmap_model(img_tensor)
      loss = predictions[:, np.argmax(predictions[0])]
      grads = gtape.gradient(loss, conv_output)
      pooled_grads = K.mean(grads, axis=(0, 1, 2))

  heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
  heatmap = np.maximum(heatmap, 0)
  max_heat = np.max(heatmap)
  if max_heat == 0:
      max_heat = 1e-10
  heatmap /= max_heat
  hm=np.squeeze(heatmap)
  plt.imshow(hm)
  if (flag == "src"):
    img = cv2.imread(image_src)
  else:
    img = img_user
  INTENSITY = 0.5

  heatmap = cv2.resize(hm, (img.shape[1], img.shape[0]))

  heatmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

  img = heatmap * INTENSITY + img
  img = np.float32(img)
  # cv2_imshow(img)
  return img

def show_heat_random():
  # plt.figure(figsize=(16,16))
  # j = 0;
  for i in os.listdir('/home/RandomCoins/'):
      # plt.subplot(4, 8, 2*j + 1)
      image1 = cv2.imread('/home/RandomCoins/'+i)
      height, width, channels = image1.shape
      image2 = do_heat_map('/home/RandomCoins/'+i, "src")

      cv2_imshow(image1)
      cv2_imshow(image2)



def show_heat_test():
  image_batch, label_batch = test_dataset.as_numpy_iterator().next()
  for i in range(9):
    image1 = (image_batch[i].astype("uint8"))
    image1 = cv2.resize(image1, (128, 128), interpolation = cv2.INTER_AREA)
    cv2_imshow(cv2.cvtColor(image1, cv2.COLOR_RGB2BGR))
    image2 = do_heat_map(image_batch[i].astype("uint8"), "nosrc")
    image2 = cv2.resize(image2, (128, 128), interpolation = cv2.INTER_AREA)
    cv2_imshow(image2)



In [ ]:
show_heat_test()

In [ ]:
show_heat_random()